<h1><center>Языковые модели и генерация текста </center></h1>

1. Вероятностные модели
2. Нейросетевые модели (RNN)

#### Построим простую вероятностную языковую модель для генерации названий динозавров. 
Возьмем данные с реальными названиями:

In [3]:
!less dinos.txt

Aachenosaurus=
Aardonyx
Abdallahsaurus
Abelisaurus
Abrictosaurus
Abrosaurus
Abydosaurus
Acanthopholis
Achelousaurus
Acheroraptor
Achillesaurus
Achillobator
Acristavus
Acrocanthosaurus
Acrotholus
Actiosaurus
Adamantisaurus
Adasaurus
Adelolophus
Adeopapposaurus
Aegyptosaurus
Aeolosaurus
Aepisaurus
Aepyornithomimus
:

In [230]:
dinos = []
with open('dinos.txt', 'r') as f:
    for line in f.readlines():
        dinos.append(list(line.strip()))

dinos[:5]

[['A', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's'],
 ['A', 'a', 'r', 'd', 'o', 'n', 'y', 'x'],
 ['A', 'b', 'd', 'a', 'l', 'l', 'a', 'h', 's', 'a', 'u', 'r', 'u', 's'],
 ['A', 'b', 'e', 'l', 'i', 's', 'a', 'u', 'r', 'u', 's'],
 ['A', 'b', 'r', 'i', 'c', 't', 'o', 's', 'a', 'u', 'r', 'u', 's']]

Мы будем учить модель для биграмм, поэтому нужно будет получить список биграмм для каждого слова:

In [231]:
from nltk.util import bigrams, pad_sequence

list(bigrams(dinos[0]))

[('A', 'a'),
 ('a', 'c'),
 ('c', 'h'),
 ('h', 'e'),
 ('e', 'n'),
 ('n', 'o'),
 ('o', 's'),
 ('s', 'a'),
 ('a', 'u'),
 ('u', 'r'),
 ('r', 'u'),
 ('u', 's')]

Помимо этого, нужно сообщить модели о начале и конце слова, поэтому добавим соответствующие символы:

In [232]:
list(pad_sequence(dinos[0], 
                  pad_left=True, 
                  left_pad_symbol="<",
                  pad_right=True, 
                  right_pad_symbol='>',
                  n = 2))

['<', 'A', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's', '>']

Все это вместе можно сделать с помощью __padded_everygram_pipeline__ : этот метод создает итераторы для данных и словаря

In [234]:
from nltk.lm.preprocessing import padded_everygram_pipeline
data, vocab = padded_everygram_pipeline(2, dinos)

Выделим часть данных для тестирования модели:

In [235]:
len(dinos)

1536

In [236]:
from random import shuffle

shuffle(dinos)
dinos[:10]

[['C', 'r', 'i', 'c', 'h', 't', 'o', 'n', 'p', 'e', 'l', 't', 'a'],
 ['S', 'k', 'o', 'r', 'p', 'i', 'o', 'v', 'e', 'n', 'a', 't', 'o', 'r'],
 ['D', 'r', 'i', 'n', 'k', 'e', 'r'],
 ['P', 'r', 'o', 'y', 'a', 'n', 'd', 'u', 's', 'a', 'u', 'r', 'u', 's'],
 ['T', 'a', 's', 't', 'a', 'v', 'i', 'n', 's', 'a', 'u', 'r', 'u', 's'],
 ['D', 'a', 'u', 'r', 'o', 's', 'a', 'u', 'r', 'u', 's'],
 ['K', 'u', 'n', 'm', 'i', 'n', 'g', 'o', 's', 'a', 'u', 'r', 'u', 's'],
 ['C', 'a', 'l', 'a', 'm', 'o', 's', 'p', 'o', 'n', 'd', 'y', 'l', 'u', 's'],
 ['R', 'u', 'b', 'e', 'o', 's', 'a', 'u', 'r', 'u', 's'],
 ['D', 'a', 'x', 'i', 'a', 't', 'i', 't', 'a', 'n']]

In [444]:
spl = int(98*len(dinos)/100)
train = dinos[:spl]
test = dinos[spl:]
len(train), len(test)

(1505, 31)

In [445]:
train_data, vocab = padded_everygram_pipeline(2, train)

Теперь можно обучать модель:

In [446]:
from nltk.lm import MLE

lm = MLE(2) # 2 = наибольший размер используемых n-грамм
len(lm.vocab)

0

In [447]:
lm.fit(train_data, vocab)
len(lm.vocab)

55

In [449]:
print(*sorted(list(lm.vocab)))

</s> <UNK> <s> A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z


Абсолютная частота униграммы:

In [370]:
lm.counts['A']

162

Абсолютная совместная частота биграммы 'Ab':

In [371]:
lm.counts[['A']]['b']

5

Частота (вероятность) униграммы:

In [372]:
lm.score('A')

0.007706945765937202

Частота (вероятность) биграмм 'bA' и 'Ab':

In [373]:
lm.score("A", ["b"]), lm.score("b", ["A"])

(0.0, 0.030864197530864196)

Можно генериовать строки заданной длины:

In [374]:
lm.generate(1, random_seed=42)

'o'

In [375]:
lm.generate(6, random_seed=42)

['o', 'c', 'e', 'l', 'o', 's']

In [376]:
lm.generate(10, random_seed=34)

['n', 'k', 'u', 's', 'h', 'i', 't', 'i', 'e', 'g']

И посчитать перплексию можели на тестовом множестве:

In [452]:
test_data = [('<s>', 'A'), ('D', 'a')]

lm.perplexity(test_data)

5.902702536398186

Обратите внимание, что перплексия для биграмм, которые не встречались в обучающией выборке, будет ожидаемо бесконечной:

In [489]:
print(lm.perplexity([('I', 'r')]))

inf


In [487]:
for t in train:
    t_ = ''.join(t)
    if t_.startswith('I'):
        print(t_)

Issasaurus
Indosuchus
Ischioceratops
Ilokelesia
Iuticosaurus
Iliosuchus
Illustration
Indosaurus
Incisivosaurus
Itemirus
Isanosaurus
Iguanodon
Iguanosaurus
Isaberrysaura
Ischisaurus
Ignavusaurus
Isisaurus
Ischyrosaurus
Iguanacolossus
Inosaurus
Iguanoides


In [378]:
test[:3]

[['Q', 'i', 'a', 'o', 'w', 'a', 'n', 'l', 'o', 'n', 'g'],
 ['T', 'o', 'n', 'g', 'a', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's'],
 ['C', 'a', 'l', 'l', 'o', 'v', 'o', 's', 'a', 'u', 'r', 'u', 's']]

In [456]:
padded_test = [['<s>']+t+['</s>'] for t in test]
print(*padded_test[:3], sep='\n')

['<s>', 'Q', 'i', 'a', 'o', 'w', 'a', 'n', 'l', 'o', 'n', 'g', '</s>']
['<s>', 'T', 'o', 'n', 'g', 'a', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's', '</s>']
['<s>', 'C', 'a', 'l', 'l', 'o', 'v', 'o', 's', 'a', 'u', 'r', 'u', 's', '</s>']


In [458]:
bigrams_test = list(flatten([list(bigrams(t)) for t in padded_test]))
bigrams_test[:20]

[('<s>', 'Q'),
 ('Q', 'i'),
 ('i', 'a'),
 ('a', 'o'),
 ('o', 'w'),
 ('w', 'a'),
 ('a', 'n'),
 ('n', 'l'),
 ('l', 'o'),
 ('o', 'n'),
 ('n', 'g'),
 ('g', '</s>'),
 ('<s>', 'T'),
 ('T', 'o'),
 ('o', 'n'),
 ('n', 'g'),
 ('g', 'a'),
 ('a', 'n'),
 ('n', 'o'),
 ('o', 's')]

In [491]:
lm.perplexity(bigrams_test)

inf

In [430]:
lm.perplexity(padded_test[:20])

8.272095348763163

Можно немного усложнить модель и учить условные вероятности для n-грамм при n = [2,3]:

In [436]:
train[:3]

[['C', 'r', 'i', 'c', 'h', 't', 'o', 'n', 'p', 'e', 'l', 't', 'a'],
 ['S', 'k', 'o', 'r', 'p', 'i', 'o', 'v', 'e', 'n', 'a', 't', 'o', 'r'],
 ['D', 'r', 'i', 'n', 'k', 'e', 'r']]

In [437]:
train_data, vocab = padded_everygram_pipeline(3, train)


lm = MLE(3)
lm.fit(train_data, vocab)

In [438]:
lm.generate(8, random_seed=42)

['o', 'c', 'e', 'p', 'i', 'n', 's', '</s>']

In [439]:
from nltk import everygrams

padded_test = list(flatten([list(everygrams(t, 2, 3)) for t in test]))
padded_test[:10]

[('Q', 'i'),
 ('i', 'a'),
 ('a', 'o'),
 ('o', 'w'),
 ('w', 'a'),
 ('a', 'n'),
 ('n', 'l'),
 ('l', 'o'),
 ('o', 'n'),
 ('n', 'g')]

In [441]:
lm.perplexity(padded_test[:10])

13.72222331052448

#### Теперь обучим нейросетевую языковую модель, а именно - рекуррентную нейросеть (RNN).

In [3]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

Подготовим данные для обучения:

In [5]:
class DinosDataset(Dataset):
    def __init__(self, path):
        super().__init__()
        self.dinos = []
        with open(path, 'r') as f:
            for line in f.readlines():
                self.dinos.append('<'+line.strip()+'>')
        self.vocab = sorted(set(''.join(self.dinos)))
        self.vocab_size = len(self.vocab)
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.dinos[index]
        x_str = line[:-1] 
        y_str = line[1:]
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.dinos)

In [6]:
trn_ds = DinosDataset('dinos.txt')

In [7]:
trn_ds.dinos[0]

'<Aachenosaurus>'

In [8]:
trn_ds.ch_to_idx

{'<': 0,
 '>': 1,
 'A': 2,
 'B': 3,
 'C': 4,
 'D': 5,
 'E': 6,
 'F': 7,
 'G': 8,
 'H': 9,
 'I': 10,
 'J': 11,
 'K': 12,
 'L': 13,
 'M': 14,
 'N': 15,
 'O': 16,
 'P': 17,
 'Q': 18,
 'R': 19,
 'S': 20,
 'T': 21,
 'U': 22,
 'V': 23,
 'W': 24,
 'X': 25,
 'Y': 26,
 'Z': 27,
 'a': 28,
 'b': 29,
 'c': 30,
 'd': 31,
 'e': 32,
 'f': 33,
 'g': 34,
 'h': 35,
 'i': 36,
 'j': 37,
 'k': 38,
 'l': 39,
 'm': 40,
 'n': 41,
 'o': 42,
 'p': 43,
 'q': 44,
 'r': 45,
 's': 46,
 't': 47,
 'u': 48,
 'v': 49,
 'w': 50,
 'x': 51,
 'y': 52,
 'z': 53}

In [9]:
trn_dl = DataLoader(trn_ds, shuffle=True)

In [10]:
x, y = trn_ds[0]
x, y

(tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0.

In [11]:
x.shape, y.shape

(torch.Size([14, 54]), torch.Size([14]))

Опишем модель, функцию потерь и алгоритм оптимизации:

In [12]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.2)
        self.i2o = nn.Linear(hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) 
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(h)
        return h, y

In [27]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [14]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['>']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != newline_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
            
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

In [15]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end ='') for x in sample_idxs]
    print()

In [28]:
def train_one_epoch(model, loss_fn, optimizer, to_print = False):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
#         if to_print:    
#             if (line_num+1) % 100 == 0:
#                 print_sample(sample(model))
           
        loss.backward()
        optimizer.step()
    
    if to_print:
        perplexity = torch.exp(loss)
        print(f'Perplexity:{perplexity}')  

In [29]:
def train(model, loss_fn, optimizer, epochs=1):
    for e in range(1, epochs+1):
        to_print = (e % 10 == 0)
        if to_print:
            print('\nEpoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer, to_print)

In [30]:
train(model, loss_fn, optimizer, epochs = 200)


Epoch:10
Perplexity:440756187693056.0

Epoch:20
Perplexity:36634997293056.0

Epoch:30
Perplexity:9922272.0

Epoch:40
Perplexity:344076960.0

Epoch:50
Perplexity:516218304.0

Epoch:60
Perplexity:11184992.0

Epoch:70
Perplexity:17884070.0

Epoch:80
Perplexity:18009483264.0

Epoch:90
Perplexity:656356868096.0

Epoch:100
Perplexity:414842112.0

Epoch:110
Perplexity:103853466845184.0

Epoch:120
Perplexity:6808408064.0

Epoch:130
Perplexity:15185391616.0

Epoch:140
Perplexity:7774433.5

Epoch:150
Perplexity:62665492.0

Epoch:160
Perplexity:5.847457528073421e+16

Epoch:170
Perplexity:17389262848.0

Epoch:180
Perplexity:8231828992.0

Epoch:190
Perplexity:144977157947392.0

Epoch:200
Perplexity:10647029760.0


In [412]:
print_sample(sample(model))

<Crgangsaurus>
